In [11]:
import requests
import pandas as pd
from lxml import html
import re
%load_ext autoreload
%autoreload 2
from utils import *
from subprocess import call
from fake_useragent import UserAgent
import requests

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
API_KEY = '34498c52b5674a6014154edbaa30d2739f265da7'

In [21]:
code = '005300'

In [22]:
start_date = '19560101'
bsn_tp = 'A001'

In [23]:
url = "http://dart.fss.or.kr/api/search.json?auth="+API_KEY+"&crp_cd="+code+"&start_dt="+start_date+"&bsn_tp="+bsn_tp+"&fin_rpt=Y&page_set=100"

In [24]:
ua = UserAgent()
headers = {'User-Agent':ua.chrome}

In [25]:
url

'http://dart.fss.or.kr/api/search.json?auth=34498c52b5674a6014154edbaa30d2739f265da7&crp_cd=005300&start_dt=19560101&bsn_tp=A001&fin_rpt=Y&page_set=100'

In [9]:
import json

In [16]:
a = requests.get(url,headers=headers).json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
urldict = {}
for row in a['list']:
    url2 = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo="
    name = row['rpt_nm']
    #Getting rid of the pre-amble that's irrelevant
    #[기재정정] [첨부추가] [첨부정정] 등 앞에 붙은 것을 제거해봅시다
    if name.find('[') != -1:
        name = name.split(']')[1]
    urldict[name] = url2+row['rcp_no']
    print(name+": "+url2+row['rcp_no'])

---

In [2]:
import requests
import pandas as pd
import io
import zipfile
import xml.etree.ElementTree as et
import json

In [11]:
def get_corpcode(crfc_key):
    params = {'crfc_key':crfc_key}
    items = ['corp_code','corp_name','stock_code','modify_date']
    item_names = ['고유번호','회사명','종목코드','수정일']
    url = 'https://opendart.fss.or.kr/api/corpCode.xml'
    res = requests.get(url, params=params)
    print(res)
    zfile = zipfile.ZipFile(io.BytesIO(res.content))
    print(zfile)
    fin = zfile.open(zfile.namelist()[0])
    root = et.fromstring(fin.read().decode('utf-8'))
    data = []
    for child in root:
        if len(child.find('stock_code').text.strip())>1:
            data.append([])
            for item in items:
                data[-1].append(child.find(item).text)
    df = pd.DataFrame(data,columns=item_names)
    return df

In [12]:
get_corpcode('34498c52b5674a6014154edbaa30d2739f265da7')

<Response [200]>


BadZipFile: File is not a zip file